### References:
    https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=cifar
    https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

In [1]:
import torch
import torchvision

In [59]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [60]:
model = ResNet(BasicBlock, [13, 9, 3, 1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

4995754

In [4]:
GPU_BATCH_SIZE_LIMIT = 512

def train(model, criterion, optimizer, dataloader, device, scheduler = None, use_scheduler = False, batch_size = 32):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
        
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                optimizer.step()
                optimizer.zero_grad()
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            loss.backward()
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, label)

            train_loss += loss.item()

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()

            loss.backward()
            
            optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

        if use_scheduler:
            scheduler.step()

    accuracy = correct/total
    train_loss = train_loss/len(dataloader)

    return train_loss, accuracy

def test(model, criterion, dataloader, device, batch_size = 32):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            output = model(image)
            loss = criterion(output, label)

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Experiment 1

In [25]:
batch_size = 32

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.7667; Training Accuracy: 50.012%
	Testing Loss: 1.0856; Testing Accuracy: 60.96%

	Epoch: 1
	Training Loss: 1.8884; Training Accuracy: 66.552%
	Testing Loss: 0.866; Testing Accuracy: 69.59%

	Epoch: 2
	Training Loss: 1.4736; Training Accuracy: 73.988%
	Testing Loss: 0.7729; Testing Accuracy: 72.5%

	Epoch: 3
	Training Loss: 1.1534; Training Accuracy: 79.998%
	Testing Loss: 0.6941; Testing Accuracy: 75.57%

	Epoch: 4
	Training Loss: 0.9014; Training Accuracy: 84.61%
	Testing Loss: 0.6739; Testing Accuracy: 77.01%


### Experiment 2

In [26]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.9655; Training Accuracy: 45.326%
	Testing Loss: 1.1382; Testing Accuracy: 58.93%

	Epoch: 1
	Training Loss: 2.143; Training Accuracy: 61.744%
	Testing Loss: 1.0245; Testing Accuracy: 64.76%

	Epoch: 2
	Training Loss: 1.7926; Training Accuracy: 68.48%
	Testing Loss: 0.9324; Testing Accuracy: 67.98%

	Epoch: 3
	Training Loss: 1.5641; Training Accuracy: 72.524%
	Testing Loss: 0.7302; Testing Accuracy: 74.19%

	Epoch: 4
	Training Loss: 1.3962; Training Accuracy: 75.616%
	Testing Loss: 0.6483; Testing Accuracy: 77.49%


### Experiment 3

In [29]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.8723; Training Accuracy: 47.636%
	Testing Loss: 1.2933; Testing Accuracy: 56.78%

	Epoch: 1
	Training Loss: 1.9635; Training Accuracy: 65.19%
	Testing Loss: 0.9888; Testing Accuracy: 66.12%

	Epoch: 2
	Training Loss: 1.5566; Training Accuracy: 73.154%
	Testing Loss: 0.7659; Testing Accuracy: 73.7%

	Epoch: 3
	Training Loss: 1.3297; Training Accuracy: 76.942%
	Testing Loss: 0.6325; Testing Accuracy: 78.42%

	Epoch: 4
	Training Loss: 1.1761; Training Accuracy: 79.468%
	Testing Loss: 0.5846; Testing Accuracy: 80.94%


### Experiment 4

In [30]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.2463; Training Accuracy: 40.35%
	Testing Loss: 1.2443; Testing Accuracy: 54.71%

	Epoch: 1
	Training Loss: 2.2736; Training Accuracy: 59.376%
	Testing Loss: 1.0165; Testing Accuracy: 65.18%

	Epoch: 2
	Training Loss: 1.8031; Training Accuracy: 68.314%
	Testing Loss: 0.8641; Testing Accuracy: 69.9%

	Epoch: 3
	Training Loss: 1.5125; Training Accuracy: 73.568%
	Testing Loss: 0.7711; Testing Accuracy: 73.79%

	Epoch: 4
	Training Loss: 1.3173; Training Accuracy: 77.168%
	Testing Loss: 0.6735; Testing Accuracy: 77.68%


### Experiment 5

In [31]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3944; Training Accuracy: 37.364%
	Testing Loss: 1.4196; Testing Accuracy: 48.3%

	Epoch: 1
	Training Loss: 2.7294; Training Accuracy: 50.724%
	Testing Loss: 1.3531; Testing Accuracy: 56.6%

	Epoch: 2
	Training Loss: 2.4085; Training Accuracy: 57.072%
	Testing Loss: 1.1211; Testing Accuracy: 61.84%

	Epoch: 3
	Training Loss: 2.1639; Training Accuracy: 61.616%
	Testing Loss: 0.9121; Testing Accuracy: 68.48%

	Epoch: 4
	Training Loss: 1.9941; Training Accuracy: 64.97%
	Testing Loss: 0.9682; Testing Accuracy: 66.47%


### Experiment 6

In [32]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3209; Training Accuracy: 38.834%
	Testing Loss: 1.3701; Testing Accuracy: 51.43%

	Epoch: 1
	Training Loss: 2.6639; Training Accuracy: 51.722%
	Testing Loss: 1.3112; Testing Accuracy: 55.95%

	Epoch: 2
	Training Loss: 2.3437; Training Accuracy: 58.34%
	Testing Loss: 1.0078; Testing Accuracy: 65.06%

	Epoch: 3
	Training Loss: 2.1284; Training Accuracy: 62.414%
	Testing Loss: 0.9128; Testing Accuracy: 68.87%

	Epoch: 4
	Training Loss: 1.9559; Training Accuracy: 65.542%
	Testing Loss: 0.7819; Testing Accuracy: 72.9%


### Experiment 7

In [34]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.642; Training Accuracy: 32.844%
	Testing Loss: 1.4509; Testing Accuracy: 46.77%

	Epoch: 1
	Training Loss: 2.9684; Training Accuracy: 45.978%
	Testing Loss: 1.4119; Testing Accuracy: 53.14%

	Epoch: 2
	Training Loss: 2.5526; Training Accuracy: 54.442%
	Testing Loss: 1.1424; Testing Accuracy: 61.25%

	Epoch: 3
	Training Loss: 2.2957; Training Accuracy: 59.394%
	Testing Loss: 1.2255; Testing Accuracy: 60.57%

	Epoch: 4
	Training Loss: 2.1109; Training Accuracy: 62.856%
	Testing Loss: 0.9835; Testing Accuracy: 67.76%


### Experiment 8

In [33]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.0835; Training Accuracy: 43.502%
	Testing Loss: 1.1675; Testing Accuracy: 58.22%

	Epoch: 1
	Training Loss: 2.3042; Training Accuracy: 58.644%
	Testing Loss: 1.1658; Testing Accuracy: 60.2%

	Epoch: 2
	Training Loss: 1.9298; Training Accuracy: 65.832%
	Testing Loss: 0.9036; Testing Accuracy: 69.68%

	Epoch: 3
	Training Loss: 1.6952; Training Accuracy: 70.314%
	Testing Loss: 0.8469; Testing Accuracy: 72.64%

	Epoch: 4
	Training Loss: 1.5312; Training Accuracy: 73.14%
	Testing Loss: 0.7011; Testing Accuracy: 76.33%


### Experiment 9

In [35]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.9925; Training Accuracy: 45.42%
	Testing Loss: 1.1651; Testing Accuracy: 57.68%

	Epoch: 1
	Training Loss: 2.1577; Training Accuracy: 61.398%
	Testing Loss: 0.9998; Testing Accuracy: 64.76%

	Epoch: 2
	Training Loss: 1.8073; Training Accuracy: 68.032%
	Testing Loss: 0.8095; Testing Accuracy: 72.64%

	Epoch: 3
	Training Loss: 1.5712; Training Accuracy: 72.424%
	Testing Loss: 0.6548; Testing Accuracy: 77.54%

	Epoch: 4
	Training Loss: 1.412; Training Accuracy: 75.32%
	Testing Loss: 0.6788; Testing Accuracy: 77.39%


### Experiment 10

In [36]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.1113; Training Accuracy: 42.69%
	Testing Loss: 1.3343; Testing Accuracy: 53.07%

	Epoch: 1
	Training Loss: 2.3764; Training Accuracy: 57.158%
	Testing Loss: 1.1477; Testing Accuracy: 61.64%

	Epoch: 2
	Training Loss: 2.0474; Training Accuracy: 63.578%
	Testing Loss: 0.9351; Testing Accuracy: 67.49%

	Epoch: 3
	Training Loss: 1.7978; Training Accuracy: 68.458%
	Testing Loss: 0.7166; Testing Accuracy: 75.16%

	Epoch: 4
	Training Loss: 1.6213; Training Accuracy: 71.424%
	Testing Loss: 0.6531; Testing Accuracy: 77.36%


### Experiment 11

In [37]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.302; Training Accuracy: 39.422%
	Testing Loss: 1.254; Testing Accuracy: 53.81%

	Epoch: 1
	Training Loss: 2.5051; Training Accuracy: 54.63%
	Testing Loss: 1.0566; Testing Accuracy: 63.47%

	Epoch: 2
	Training Loss: 2.1467; Training Accuracy: 61.798%
	Testing Loss: 0.9029; Testing Accuracy: 68.43%

	Epoch: 3
	Training Loss: 1.8728; Training Accuracy: 66.908%
	Testing Loss: 0.884; Testing Accuracy: 70.27%

	Epoch: 4
	Training Loss: 1.6656; Training Accuracy: 70.648%
	Testing Loss: 0.6646; Testing Accuracy: 77.54%


### Experiment 12

In [38]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.075, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.4; Training Accuracy: 36.974%
	Testing Loss: 1.3463; Testing Accuracy: 49.1%

	Epoch: 1
	Training Loss: 2.62; Training Accuracy: 52.444%
	Testing Loss: 1.1415; Testing Accuracy: 59.94%

	Epoch: 2
	Training Loss: 2.218; Training Accuracy: 60.346%
	Testing Loss: 0.9475; Testing Accuracy: 66.81%

	Epoch: 3
	Training Loss: 1.975; Training Accuracy: 65.116%
	Testing Loss: 0.8218; Testing Accuracy: 71.99%

	Epoch: 4
	Training Loss: 1.7621; Training Accuracy: 69.124%
	Testing Loss: 0.742; Testing Accuracy: 73.55%


### Experiment 13

In [39]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3515; Training Accuracy: 38.206%
	Testing Loss: 1.458; Testing Accuracy: 48.59%

	Epoch: 1
	Training Loss: 2.5859; Training Accuracy: 52.998%
	Testing Loss: 1.1997; Testing Accuracy: 57.94%

	Epoch: 2
	Training Loss: 2.2156; Training Accuracy: 60.366%
	Testing Loss: 0.9809; Testing Accuracy: 65.84%

	Epoch: 3
	Training Loss: 1.9713; Training Accuracy: 65.284%
	Testing Loss: 0.8788; Testing Accuracy: 70.32%

	Epoch: 4
	Training Loss: 1.7682; Training Accuracy: 68.818%
	Testing Loss: 0.7629; Testing Accuracy: 72.58%


### Experiment 14

In [40]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.04, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3225; Training Accuracy: 38.332%
	Testing Loss: 1.3154; Testing Accuracy: 53.49%

	Epoch: 1
	Training Loss: 2.5591; Training Accuracy: 53.814%
	Testing Loss: 1.1136; Testing Accuracy: 61.09%

	Epoch: 2
	Training Loss: 2.1821; Training Accuracy: 61.154%
	Testing Loss: 0.9609; Testing Accuracy: 67.25%

	Epoch: 3
	Training Loss: 1.9458; Training Accuracy: 65.308%
	Testing Loss: 0.9409; Testing Accuracy: 68.38%

	Epoch: 4
	Training Loss: 1.7446; Training Accuracy: 69.02%
	Testing Loss: 0.8189; Testing Accuracy: 70.96%


### Experiment 15

In [41]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.055, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3325; Training Accuracy: 38.806%
	Testing Loss: 1.3085; Testing Accuracy: 51.63%

	Epoch: 1
	Training Loss: 2.5593; Training Accuracy: 53.708%
	Testing Loss: 1.0251; Testing Accuracy: 63.24%

	Epoch: 2
	Training Loss: 2.1877; Training Accuracy: 60.898%
	Testing Loss: 1.0142; Testing Accuracy: 65.27%

	Epoch: 3
	Training Loss: 1.9567; Training Accuracy: 65.5%
	Testing Loss: 0.8049; Testing Accuracy: 71.62%

	Epoch: 4
	Training Loss: 1.7464; Training Accuracy: 69.264%
	Testing Loss: 0.8074; Testing Accuracy: 73.08%


### Experiment 16

In [42]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.2998; Training Accuracy: 39.026%
	Testing Loss: 1.4011; Testing Accuracy: 49.13%

	Epoch: 1
	Training Loss: 2.5151; Training Accuracy: 54.708%
	Testing Loss: 1.1535; Testing Accuracy: 60.21%

	Epoch: 2
	Training Loss: 2.1415; Training Accuracy: 61.882%
	Testing Loss: 0.8952; Testing Accuracy: 68.29%

	Epoch: 3
	Training Loss: 1.8932; Training Accuracy: 66.546%
	Testing Loss: 0.9351; Testing Accuracy: 68.3%

	Epoch: 4
	Training Loss: 1.6797; Training Accuracy: 70.32%
	Testing Loss: 0.7241; Testing Accuracy: 75.16%


### Experiment 17

In [43]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0005)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3636; Training Accuracy: 38.234%
	Testing Loss: 1.3329; Testing Accuracy: 51.38%

	Epoch: 1
	Training Loss: 2.6387; Training Accuracy: 52.36%
	Testing Loss: 1.2368; Testing Accuracy: 56.9%

	Epoch: 2
	Training Loss: 2.2993; Training Accuracy: 58.896%
	Testing Loss: 1.0868; Testing Accuracy: 63.22%

	Epoch: 3
	Training Loss: 2.1267; Training Accuracy: 62.364%
	Testing Loss: 0.939; Testing Accuracy: 67.81%

	Epoch: 4
	Training Loss: 1.968; Training Accuracy: 65.744%
	Testing Loss: 0.9173; Testing Accuracy: 69.4%


### Experiment 18

In [44]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0002)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.3376; Training Accuracy: 38.508%
	Testing Loss: 1.366; Testing Accuracy: 50.21%

	Epoch: 1
	Training Loss: 2.5915; Training Accuracy: 53.026%
	Testing Loss: 1.1001; Testing Accuracy: 60.88%

	Epoch: 2
	Training Loss: 2.2289; Training Accuracy: 60.142%
	Testing Loss: 1.0309; Testing Accuracy: 63.19%

	Epoch: 3
	Training Loss: 1.9933; Training Accuracy: 64.85%
	Testing Loss: 0.9037; Testing Accuracy: 68.69%

	Epoch: 4
	Training Loss: 1.777; Training Accuracy: 68.81%
	Testing Loss: 0.7885; Testing Accuracy: 73.21%


### Experiment 19

In [45]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.2079; Training Accuracy: 41.196%
	Testing Loss: 1.2863; Testing Accuracy: 54.28%

	Epoch: 1
	Training Loss: 2.4307; Training Accuracy: 56.526%
	Testing Loss: 1.0198; Testing Accuracy: 64.29%

	Epoch: 2
	Training Loss: 2.092; Training Accuracy: 62.956%
	Testing Loss: 0.9101; Testing Accuracy: 67.67%

	Epoch: 3
	Training Loss: 1.855; Training Accuracy: 67.366%
	Testing Loss: 0.7747; Testing Accuracy: 72.99%

	Epoch: 4
	Training Loss: 1.6575; Training Accuracy: 71.082%
	Testing Loss: 0.697; Testing Accuracy: 76.1%


### Experiment 20

In [46]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.2406; Training Accuracy: 40.342%
	Testing Loss: 1.3901; Testing Accuracy: 52.03%

	Epoch: 1
	Training Loss: 2.443; Training Accuracy: 55.966%
	Testing Loss: 1.0469; Testing Accuracy: 61.85%

	Epoch: 2
	Training Loss: 2.0991; Training Accuracy: 62.636%
	Testing Loss: 0.8484; Testing Accuracy: 69.9%

	Epoch: 3
	Training Loss: 1.8708; Training Accuracy: 66.786%
	Testing Loss: 0.7403; Testing Accuracy: 74.14%

	Epoch: 4
	Training Loss: 1.6816; Training Accuracy: 70.416%
	Testing Loss: 0.7146; Testing Accuracy: 75.79%

	Epoch: 5
	Training Loss: 1.539; Training Accuracy: 73.148%
	Testing Loss: 0.6257; Testing Accuracy: 78.99%

	Epoch: 6
	Training Loss: 1.4279; Training Accuracy: 74.998%
	Testing Loss: 0.6519; Testing Accuracy: 77.7%

	Epoch: 7
	Training Loss: 1.3414; Training Accuracy: 76.54%
	Testing Loss: 0.5664; Testing Accuracy: 81.08%

	Epoch: 8
	Training Loss: 1

### Experiment 21

In [47]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 64)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 64)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.134; Training Accuracy: 42.594%
	Testing Loss: 1.258; Testing Accuracy: 53.98%

	Epoch: 1
	Training Loss: 2.3799; Training Accuracy: 56.926%
	Testing Loss: 1.0215; Testing Accuracy: 64.19%

	Epoch: 2
	Training Loss: 2.0249; Training Accuracy: 64.034%
	Testing Loss: 0.9987; Testing Accuracy: 66.57%

	Epoch: 3
	Training Loss: 1.8068; Training Accuracy: 68.006%
	Testing Loss: 0.7703; Testing Accuracy: 73.37%

	Epoch: 4
	Training Loss: 1.6042; Training Accuracy: 71.846%
	Testing Loss: 0.7841; Testing Accuracy: 73.34%

	Epoch: 5
	Training Loss: 1.4742; Training Accuracy: 73.964%
	Testing Loss: 0.6081; Testing Accuracy: 79.2%

	Epoch: 6
	Training Loss: 1.3671; Training Accuracy: 76.096%
	Testing Loss: 0.6307; Testing Accuracy: 79.78%

	Epoch: 7
	Training Loss: 1.2678; Training Accuracy: 77.608%
	Testing Loss: 0.5951; Testing Accuracy: 80.1%

	Epoch: 8
	Training Loss: 

### Detailed Analysis

In [57]:
batch_sizes    = [32, 64, 128]
augmentations  = ['no', 'random_crop', 'random_horizontal_flip', 'random_rotate', 
                  'random_resized_crop', 'random_affine', 'random_erasing']
optimizers     = ['SGD', 'SGD with Nesterov', 'ASGD', 'Adam', 'Adamax', 'Adadelta']

metrics_dict = {}

for batch_size in batch_sizes:
    metrics_dict[batch_size] = {}
    for augmentation in augmentations:
        metrics_dict[batch_size][augmentation] = {}
        for optimizer_id in optimizers:
            metrics_dict[batch_size][augmentation][optimizer_id] = {}
            
            torch.cuda.empty_cache()
            model = ResNet(BasicBlock, [1, 1, 1, 1])
            num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            model.to(device)
            criterion = nn.CrossEntropyLoss()

            print("\n\n")
            print("=========================================================================================================")
            print(f"Batch Size: {batch_size}; Augmentation: {augmentation}; Optimizer: {optimizer_id}; Num Params: {num_params}\n")

            train_losses = []
            val_losses = []
            train_accuracies = []
            val_accuracies = []

            try:
                if augmentation == 'no':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_crop':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.RandomCrop(32, padding=4),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_horizontal_flip':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.RandomHorizontalFlip(),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_rotate':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.RandomRotation(15),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_resized_crop':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_affine':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                elif augmentation == 'random_erasing':
                    transform_train = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                        torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))])

                transform_test = torchvision.transforms.Compose([
                                    torchvision.transforms.ToTensor(), 
                                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

                trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
                trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

                testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
                testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

                lr_min = 1e-4
                lr_max = 1e-1

                if optimizer_id == 'SGD':
                    optimizer = torch.optim.SGD(model.parameters(), lr_min, momentum=0.9, weight_decay=0.0001)
                    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, lr_min, lr_max, step_size_up=10000, gamma=0.85, cycle_momentum=True)
                elif optimizer_id == 'SGD with Nesterov':
                    optimizer = torch.optim.SGD(model.parameters(), lr_min, momentum=0.9, weight_decay=0.0001, nesterov=True)
                    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, lr_min, lr_max, step_size_up=10000, gamma=0.85, cycle_momentum=True)
                else:
                    if optimizer_id == 'ASGD':
                        optimizer = torch.optim.ASGD(model.parameters(), lr_min, weight_decay=0.0001)
                    elif optimizer_id == 'Adam':
                        optimizer = torch.optim.Adam(model.parameters(), lr_min, weight_decay=0.0001)
                    elif optimizer_id == 'Adamax':
                        optimizer = torch.optim.Adamax(model.parameters(), lr_min, weight_decay=0.0001)
                    elif optimizer_id == 'Adadelta':
                        optimizer = torch.optim.Adadelta(model.parameters(), lr_min, weight_decay=0.0001)
                    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, lr_min, lr_max, step_size_up=10000, gamma=0.85, cycle_momentum=False)

                epochs = 5
                for epoch in range(epochs):
                    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, scheduler, True, batch_size = batch_size)
                    train_losses.append(train_loss)
                    train_accuracies.append(train_accuracy)
                    val_loss, val_accuracy = test(model, criterion, testloader, device, batch_size = batch_size)
                    val_losses.append(val_loss)
                    val_accuracies.append(val_accuracy)
                    print(f"\n\tEpoch: {epoch}")
                    print(f"\tTraining Loss   : {round(train_loss, 4)}; Training Accuracy   : {round(train_accuracy*100, 4)}%")
                    print(f"\tValidation Loss : {round(val_loss, 4)}; Validation Accuracy : {round(val_accuracy*100, 4)}%")
            except Exception as e:
                metrics_dict[batch_size][augmentation][optimizer_id]['error'] = str(e)
                print(f"\tError: {str(e)}")
            metrics_dict[batch_size][augmentation][optimizer_id]['train_loss'] = train_losses
            metrics_dict[batch_size][augmentation][optimizer_id]['train_acc'] = train_accuracies
            metrics_dict[batch_size][augmentation][optimizer_id]['valid_loss'] = val_losses
            metrics_dict[batch_size][augmentation][optimizer_id]['valid_acc'] = val_accuracies
            print("=========================================================================================================")




Batch Size: 32; Augmentation: no; Optimizer: SGD; Num Params: 4903242

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss   : 2.8012; Training Accuracy   : 49.294%
	Validation Loss : 1.2899; Validation Accuracy : 56.95%

	Epoch: 1
	Training Loss   : 1.775; Training Accuracy   : 68.76%
	Validation Loss : 0.7429; Validation Accuracy : 74.31%

	Epoch: 2
	Training Loss   : 1.3327; Training Accuracy   : 76.552%
	Validation Loss : 0.6936; Validation Accuracy : 76.33%

	Epoch: 3
	Training Loss   : 1.0503; Training Accuracy   : 81.54%
	Validation Loss : 0.6543; Validation Accuracy : 77.69%

	Epoch: 4
	Training Loss   : 0.8564; Training Accuracy   : 85.174%
	Validation Loss : 0.5525; Validation Accuracy : 81.19%



Batch Size: 32; Augmentation: no; Optimizer: SGD with Nesterov; Num Params: 4903242

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss   : 2.6844; Training Accuracy   : 51.362%
	

In [58]:
import json

json_obj = json.dumps(metrics_dict, indent = 4)

with open("metrics.json", "w") as outfile:
    json.dump(json_obj, outfile)

In [61]:
rows = []
for b_s in list(metrics_dict.keys()):
    for aug in list(metrics_dict[b_s].keys()):
        for opt in list(metrics_dict[b_s][aug].keys()):
            train_loss = metrics_dict[b_s][aug][opt]['train_loss'][-1]
            train_acc = metrics_dict[b_s][aug][opt]['train_acc'][-1]*100
            valid_loss = metrics_dict[b_s][aug][opt]['valid_loss'][-1]
            valid_acc = metrics_dict[b_s][aug][opt]['valid_acc'][-1]*100
            row = [b_s, aug, opt, train_loss, valid_loss, train_acc, valid_acc]
            rows.append(row)

In [63]:
import pandas as pd

columns = ['batch_size', 'augmentation', 'optimizer', 'train_loss', 'valid_loss', 'train_acc', 'valid_acc']
df = pd.DataFrame(rows, columns = columns)
df.to_csv('metrics.csv')
df

,batch_size,augmentation,optimizer,train_loss,valid_loss,train_acc,valid_acc
0,32,no,SGD,0.856368,0.552547,85.174,81.19
1,32,no,SGD with Nesterov,0.824306,0.551144,85.348,81.77
2,32,no,ASGD,1.049168,0.810727,81.768,73.52
3,32,no,Adam,2.585961,1.659058,54.276,40.51
4,32,no,Adamax,1.840848,1.019998,67.942,64.22
...,...,...,...,...,...,...,...
121,128,random_erasing,SGD with Nesterov,1.218360,1.266651,78.466,63.24
122,128,random_erasing,ASGD,2.049126,1.058681,63.284,62.90
123,128,random_erasing,Adam,1.753814,1.043496,69.298,64.62
124,128,random_erasing,Adamax,1.449330,0.801869,74.776,72.80


In [64]:
df[df['batch_size'] == 32][df['optimizer'] == 'SGD with Nesterov']

C:\Users\suriy\AppData\Local\Temp\ipykernel_8772\1658992418.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['batch_size'] == 32][df['optimizer'] == 'SGD with Nesterov']


,batch_size,augmentation,optimizer,train_loss,valid_loss,train_acc,valid_acc
1,32,no,SGD with Nesterov,0.824306,0.551144,85.348,81.77
7,32,random_crop,SGD with Nesterov,1.147717,0.578310,80.240,80.63
13,32,random_horizontal_flip,SGD with Nesterov,0.946680,0.597674,83.478,79.60
19,32,random_rotate,SGD with Nesterov,1.161062,0.609516,79.840,78.65
25,32,random_resized_crop,SGD with Nesterov,1.084598,0.552147,81.040,80.83
31,32,random_affine,SGD with Nesterov,1.375324,0.630698,76.018,78.55
37,32,random_erasing,SGD with Nesterov,1.077594,0.710151,81.130,76.79


In [67]:
df[df['batch_size'] == 64][df['optimizer'] == 'SGD with Nesterov']

C:\Users\suriy\AppData\Local\Temp\ipykernel_8772\83037611.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['batch_size'] == 64][df['optimizer'] == 'SGD with Nesterov']


,batch_size,augmentation,optimizer,train_loss,valid_loss,train_acc,valid_acc
43,64,no,SGD with Nesterov,0.866554,0.615567,84.870,79.91
49,64,random_crop,SGD with Nesterov,1.211746,0.650072,79.008,78.16
55,64,random_horizontal_flip,SGD with Nesterov,0.995144,0.824317,82.616,73.04
61,64,random_rotate,SGD with Nesterov,1.214871,0.661693,78.810,77.58
67,64,random_resized_crop,SGD with Nesterov,1.155625,0.871453,79.786,72.45
73,64,random_affine,SGD with Nesterov,1.452721,0.714894,74.678,75.17
79,64,random_erasing,SGD with Nesterov,1.131242,0.658130,80.066,77.49


In [68]:
df[df['batch_size'] == 128][df['optimizer'] == 'SGD with Nesterov']

C:\Users\suriy\AppData\Local\Temp\ipykernel_8772\2440078975.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['batch_size'] == 128][df['optimizer'] == 'SGD with Nesterov']


,batch_size,augmentation,optimizer,train_loss,valid_loss,train_acc,valid_acc
85,128,no,SGD with Nesterov,0.969546,0.703783,83.028,76.15
91,128,random_crop,SGD with Nesterov,1.304945,0.718166,77.264,76.63
97,128,random_horizontal_flip,SGD with Nesterov,1.059241,0.692874,81.560,76.17
103,128,random_rotate,SGD with Nesterov,1.319997,0.682015,76.828,77.00
109,128,random_resized_crop,SGD with Nesterov,1.247956,0.750942,78.176,74.21
115,128,random_affine,SGD with Nesterov,1.574058,0.715928,72.208,75.37
121,128,random_erasing,SGD with Nesterov,1.218360,1.266651,78.466,63.24


### 32-Batch Best Model Test : 50 Epochs

In [72]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

lr_min = 1e-4
lr_max = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr_min, momentum=0.9, weight_decay=0.0001, nesterov=True)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, lr_min, lr_max, step_size_up=10, gamma=0.85, cycle_momentum=True)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = batch_size)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = batch_size)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.7754; Training Accuracy: 31.19%
	Testing Loss: 1.6541; Testing Accuracy: 39.56%

	Epoch: 1
	Training Loss: 3.2455; Training Accuracy: 40.714%
	Testing Loss: 1.4607; Testing Accuracy: 46.2%

	Epoch: 2
	Training Loss: 3.0015; Training Accuracy: 45.488%
	Testing Loss: 1.3887; Testing Accuracy: 49.12%

	Epoch: 3
	Training Loss: 2.8038; Training Accuracy: 49.496%
	Testing Loss: 1.3523; Testing Accuracy: 51.33%

	Epoch: 4
	Training Loss: 2.6495; Training Accuracy: 52.486%
	Testing Loss: 1.2799; Testing Accuracy: 53.91%

	Epoch: 5
	Training Loss: 2.5078; Training Accuracy: 55.354%
	Testing Loss: 1.2978; Testing Accuracy: 53.41%

	Epoch: 6
	Training Loss: 2.3791; Training Accuracy: 57.43%
	Testing Loss: 1.201; Testing Accuracy: 57.81%

	Epoch: 7
	Training Loss: 2.2689; Training Accuracy: 59.72%
	Testing Loss: 1.1018; Testing Accuracy: 60.34%

	Epoch: 8
	Training Loss: 2

In [74]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, weight_decay=0.0001, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = batch_size)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = batch_size)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.8838; Training Accuracy: 47.548%
	Testing Loss: 1.4132; Testing Accuracy: 52.52%

	Epoch: 1
	Training Loss: 2.0007; Training Accuracy: 64.462%
	Testing Loss: 0.9203; Testing Accuracy: 69.2%

	Epoch: 2
	Training Loss: 1.6315; Training Accuracy: 71.716%
	Testing Loss: 0.8155; Testing Accuracy: 73.1%

	Epoch: 3
	Training Loss: 1.4254; Training Accuracy: 75.178%
	Testing Loss: 0.6703; Testing Accuracy: 77.32%

	Epoch: 4
	Training Loss: 1.2814; Training Accuracy: 77.87%
	Testing Loss: 0.6159; Testing Accuracy: 79.04%

	Epoch: 5
	Training Loss: 1.169; Training Accuracy: 79.848%
	Testing Loss: 0.6443; Testing Accuracy: 78.62%

	Epoch: 6
	Training Loss: 1.0691; Training Accuracy: 81.342%
	Testing Loss: 0.6004; Testing Accuracy: 80.16%

	Epoch: 7
	Training Loss: 0.9985; Training Accuracy: 82.642%
	Testing Loss: 0.5193; Testing Accuracy: 82.84%

	Epoch: 8
	Training Loss: 

In [77]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.1, momentum=0.9, weight_decay=0.0001, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = batch_size)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = batch_size)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.2705; Training Accuracy: 39.896%
	Testing Loss: 1.2401; Testing Accuracy: 55.04%

	Epoch: 1
	Training Loss: 2.3197; Training Accuracy: 58.602%
	Testing Loss: 1.1184; Testing Accuracy: 60.96%

	Epoch: 2
	Training Loss: 1.9093; Training Accuracy: 66.428%
	Testing Loss: 0.8954; Testing Accuracy: 68.66%

	Epoch: 3
	Training Loss: 1.6293; Training Accuracy: 71.382%
	Testing Loss: 0.8284; Testing Accuracy: 72.17%

	Epoch: 4
	Training Loss: 1.4244; Training Accuracy: 75.22%
	Testing Loss: 0.6908; Testing Accuracy: 76.36%

	Epoch: 5
	Training Loss: 1.3082; Training Accuracy: 77.316%
	Testing Loss: 0.6786; Testing Accuracy: 77.65%

	Epoch: 6
	Training Loss: 1.2148; Training Accuracy: 79.052%
	Testing Loss: 0.6042; Testing Accuracy: 79.79%

	Epoch: 7
	Training Loss: 1.1519; Training Accuracy: 80.142%
	Testing Loss: 0.6133; Testing Accuracy: 79.58%

	Epoch: 8
	Training Los

### 64-Batch Best Model Test : 50 Epochs

In [82]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, verbose = True)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 50
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = batch_size)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = batch_size)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 2.9136; Training Accuracy: 46.844%
	Testing Loss: 1.4071; Testing Accuracy: 54.03%

	Epoch: 1
	Training Loss: 2.2236; Training Accuracy: 59.944%
	Testing Loss: 1.1614; Testing Accuracy: 60.14%

	Epoch: 2
	Training Loss: 1.9215; Training Accuracy: 65.992%
	Testing Loss: 1.0184; Testing Accuracy: 65.37%

	Epoch: 3
	Training Loss: 1.7158; Training Accuracy: 69.748%
	Testing Loss: 0.7368; Testing Accuracy: 75.07%

	Epoch: 4
	Training Loss: 1.5727; Training Accuracy: 72.308%
	Testing Loss: 0.6939; Testing Accuracy: 76.44%

	Epoch: 5
	Training Loss: 1.4431; Training Accuracy: 74.894%
	Testing Loss: 0.6965; Testing Accuracy: 76.58%

	Epoch: 6
	Training Loss: 1.3471; Training Accuracy: 76.314%
	Testing Loss: 0.6145; Testing Accuracy: 79.31%

	Epoch: 7
	Training Loss: 1.2848; Training Accuracy: 77.416%
	Testing Loss: 0.6696; Testing Accuracy: 78.26%

	Epoch: 8
	Training Lo

In [10]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = False):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)
    
    if use_scheduler:
        scheduler.step(valid_accuracy)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [62]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [13, 9, 3, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor=0.1, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                   train_loader, val_loader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")
    
test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4995754


	Epoch: 0
	Training Loss: 3.3507; Training Accuracy: 38.165%
	Validation Loss: 1.4942; Validation Accuracy: 45.93%

	Epoch: 1
	Training Loss: 2.5894; Training Accuracy: 53.0275%
	Validation Loss: 1.1336; Validation Accuracy: 59.2%

	Epoch: 2
	Training Loss: 2.1222; Training Accuracy: 62.2975%
	Validation Loss: 1.1009; Validation Accuracy: 61.96%

	Epoch: 3
	Training Loss: 1.8115; Training Accuracy: 68.4075%
	Validation Loss: 0.917; Validation Accuracy: 67.44%

	Epoch: 4
	Training Loss: 1.6177; Training Accuracy: 71.5925%
	Validation Loss: 0.8766; Validation Accuracy: 69.67%

	Epoch: 5
	Training Loss: 1.4715; Training Accuracy: 74.395%
	Validation Loss: 0.7432; Validation Accuracy: 74.06%

	Epoch: 6
	Training Loss: 1.3528; Training Accuracy: 76.4025%
	Validation Loss: 0.7266; Validation Accuracy: 74.87%

	Epoch: 7
	Training Loss: 1.2559; Training Accuracy: 78.1475%
	Validation Loss: 0.6883;

### 128-Batch Best Model Test : 50 Epochs

In [63]:
batch_size = 128

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [13, 9, 3, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor=0.1, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                   train_loader, val_loader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")
    
test_loss, test_accuracy = test(model, criterion, test_loader, device)

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4995754


	Epoch: 0
	Training Loss: 3.5703; Training Accuracy: 33.6025%
	Validation Loss: 1.5959; Validation Accuracy: 41.63%

	Epoch: 1
	Training Loss: 2.9106; Training Accuracy: 47.065%
	Validation Loss: 1.4449; Validation Accuracy: 47.88%

	Epoch: 2
	Training Loss: 2.5629; Training Accuracy: 53.7725%
	Validation Loss: 1.5744; Validation Accuracy: 46.37%

	Epoch: 3
	Training Loss: 2.3204; Training Accuracy: 58.505%
	Validation Loss: 1.2219; Validation Accuracy: 56.48%

	Epoch: 4
	Training Loss: 2.1304; Training Accuracy: 62.11%
	Validation Loss: 1.1345; Validation Accuracy: 60.74%

	Epoch: 5
	Training Loss: 1.9824; Training Accuracy: 64.895%
	Validation Loss: 1.0045; Validation Accuracy: 64.68%

	Epoch: 6
	Training Loss: 1.8727; Training Accuracy: 66.8975%
	Validation Loss: 0.9952; Validation Accuracy: 64.2%

	Epoch: 7
	Training Loss: 1.7666; Training Accuracy: 69.0125%
	Validation Loss: 0.9745; V

In [65]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [13, 9, 3, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor=0.1, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                   train_loader, val_loader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")
    
test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4995754


	Epoch: 0
	Training Loss: 3.5145; Training Accuracy: 35.37%
	Validation Loss: 1.5966; Validation Accuracy: 41.17%

	Epoch: 1
	Training Loss: 2.7155; Training Accuracy: 50.79%
	Validation Loss: 1.1834; Validation Accuracy: 57.79%

	Epoch: 2
	Training Loss: 2.1133; Training Accuracy: 62.64%
	Validation Loss: 0.9748; Validation Accuracy: 65.5%

	Epoch: 3
	Training Loss: 1.7739; Training Accuracy: 69.0225%
	Validation Loss: 0.8479; Validation Accuracy: 70.14%

	Epoch: 4
	Training Loss: 1.5581; Training Accuracy: 73.15%
	Validation Loss: 0.7715; Validation Accuracy: 73.13%

	Epoch: 5
	Training Loss: 1.4186; Training Accuracy: 75.465%
	Validation Loss: 0.7456; Validation Accuracy: 74.53%

	Epoch: 6
	Training Loss: 1.3079; Training Accuracy: 77.5175%
	Validation Loss: 0.6303; Validation Accuracy: 78.1%

	Epoch: 7
	Training Loss: 1.2202; Training Accuracy: 78.8425%
	Validation Loss: 0.6088; Valida

In [28]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
    torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4903242


	Epoch: 0
	Training Loss: 3.4431; Training Accuracy: 36.506%
	Testing Loss: 1.3897; Testing Accuracy: 49.3%

	Epoch: 1
	Training Loss: 2.8815; Training Accuracy: 48.006%
	Testing Loss: 1.1536; Testing Accuracy: 58.68%

	Epoch: 2
	Training Loss: 2.5818; Training Accuracy: 53.81%
	Testing Loss: 1.0249; Testing Accuracy: 63.46%

	Epoch: 3
	Training Loss: 2.3971; Training Accuracy: 57.468%
	Testing Loss: 1.0749; Testing Accuracy: 63.07%

	Epoch: 4
	Training Loss: 2.2375; Training Accuracy: 60.842%
	Testing Loss: 1.0195; Testing Accuracy: 65.91%


### Experiment 7

In [8]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.0695; Training Accuracy: 44.138%
	Testing Loss: 1.1521; Testing Accuracy: 59.37%

	Epoch: 1
	Training Loss: 2.0188; Training Accuracy: 64.102%
	Testing Loss: 0.8863; Testing Accuracy: 69.66%

	Epoch: 2
	Training Loss: 1.5569; Training Accuracy: 72.766%
	Testing Loss: 0.7497; Testing Accuracy: 74.66%

	Epoch: 3
	Training Loss: 1.2934; Training Accuracy: 77.598%
	Testing Loss: 0.7004; Testing Accuracy: 76.84%

	Epoch: 4
	Training Loss: 1.1314; Training Accuracy: 80.576%
	Testing Loss: 0.6563; Testing Accuracy: 77.86%


### Experiment 4

In [9]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 4.0831; Training Accuracy: 25.416%
	Testing Loss: 1.7573; Testing Accuracy: 36.93%

	Epoch: 1
	Training Loss: 3.0747; Training Accuracy: 43.314%
	Testing Loss: 1.3138; Testing Accuracy: 53.16%

	Epoch: 2
	Training Loss: 2.3891; Training Accuracy: 57.44%
	Testing Loss: 0.9893; Testing Accuracy: 64.71%

	Epoch: 3
	Training Loss: 1.8712; Training Accuracy: 67.124%
	Testing Loss: 0.9075; Testing Accuracy: 69.66%

	Epoch: 4
	Training Loss: 1.5071; Training Accuracy: 73.58%
	Testing Loss: 0.7263; Testing Accuracy: 75.88%


### Experiment 5

In [10]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.8098; Training Accuracy: 48.56%
	Testing Loss: 1.0293; Testing Accuracy: 62.74%

	Epoch: 1
	Training Loss: 1.8649; Training Accuracy: 66.91%
	Testing Loss: 0.8283; Testing Accuracy: 71.45%

	Epoch: 2
	Training Loss: 1.464; Training Accuracy: 74.628%
	Testing Loss: 0.6756; Testing Accuracy: 76.84%

	Epoch: 3
	Training Loss: 1.2396; Training Accuracy: 78.24%
	Testing Loss: 0.5828; Testing Accuracy: 79.73%

	Epoch: 4
	Training Loss: 1.092; Training Accuracy: 81.114%
	Testing Loss: 0.5457; Testing Accuracy: 81.31%


### Experiment 6

In [13]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.9954; Training Accuracy: 45.102%
	Testing Loss: 1.2489; Testing Accuracy: 56.77%

	Epoch: 1
	Training Loss: 2.0779; Training Accuracy: 63.1%
	Testing Loss: 1.0681; Testing Accuracy: 64.61%

	Epoch: 2
	Training Loss: 1.6596; Training Accuracy: 71.018%
	Testing Loss: 0.8558; Testing Accuracy: 71.83%

	Epoch: 3
	Training Loss: 1.4053; Training Accuracy: 75.928%
	Testing Loss: 0.7706; Testing Accuracy: 74.29%

	Epoch: 4
	Training Loss: 1.2425; Training Accuracy: 78.668%
	Testing Loss: 0.6392; Testing Accuracy: 78.47%


### Experiment 7

In [14]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.001, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.6369; Training Accuracy: 52.344%
	Testing Loss: 1.0153; Testing Accuracy: 63.82%

	Epoch: 1
	Training Loss: 1.8463; Training Accuracy: 67.354%
	Testing Loss: 0.8102; Testing Accuracy: 71.3%

	Epoch: 2
	Training Loss: 1.5579; Training Accuracy: 72.572%
	Testing Loss: 0.7208; Testing Accuracy: 75.06%

	Epoch: 3
	Training Loss: 1.3769; Training Accuracy: 75.914%
	Testing Loss: 0.6677; Testing Accuracy: 76.91%

	Epoch: 4
	Training Loss: 1.249; Training Accuracy: 78.262%
	Testing Loss: 0.6503; Testing Accuracy: 77.46%


### Experiment 8

In [15]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.9072; Training Accuracy: 27.728%
	Testing Loss: 1.6921; Testing Accuracy: 37.86%

	Epoch: 1
	Training Loss: 3.3217; Training Accuracy: 39.254%
	Testing Loss: 1.5291; Testing Accuracy: 43.78%

	Epoch: 2
	Training Loss: 3.0834; Training Accuracy: 43.614%
	Testing Loss: 1.4312; Testing Accuracy: 46.77%

	Epoch: 3
	Training Loss: 2.9362; Training Accuracy: 46.198%
	Testing Loss: 1.3586; Testing Accuracy: 49.49%

	Epoch: 4
	Training Loss: 2.8134; Training Accuracy: 48.944%
	Testing Loss: 1.3126; Testing Accuracy: 52.6%


### Experiment 9

In [17]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 64)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 64)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.9782; Training Accuracy: 45.238%
	Testing Loss: 1.2115; Testing Accuracy: 55.69%

	Epoch: 1
	Training Loss: 2.0646; Training Accuracy: 63.354%
	Testing Loss: 0.907; Testing Accuracy: 68.15%

	Epoch: 2
	Training Loss: 1.6231; Training Accuracy: 71.274%
	Testing Loss: 0.7754; Testing Accuracy: 73.4%

	Epoch: 3
	Training Loss: 1.3706; Training Accuracy: 75.896%
	Testing Loss: 0.6829; Testing Accuracy: 76.34%

	Epoch: 4
	Training Loss: 1.2008; Training Accuracy: 79.018%
	Testing Loss: 0.6093; Testing Accuracy: 79.54%


### Experiment 10

In [18]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 64)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 64)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.9506; Training Accuracy: 45.832%
	Testing Loss: 1.1447; Testing Accuracy: 59.1%

	Epoch: 1
	Training Loss: 2.0538; Training Accuracy: 63.166%
	Testing Loss: 0.9469; Testing Accuracy: 66.62%

	Epoch: 2
	Training Loss: 1.6601; Training Accuracy: 70.54%
	Testing Loss: 0.7957; Testing Accuracy: 72.57%

	Epoch: 3
	Training Loss: 1.4186; Training Accuracy: 75.384%
	Testing Loss: 0.7268; Testing Accuracy: 74.97%

	Epoch: 4
	Training Loss: 1.2425; Training Accuracy: 78.384%
	Testing Loss: 0.6675; Testing Accuracy: 77.2%

	Epoch: 5
	Training Loss: 1.111; Training Accuracy: 80.596%
	Testing Loss: 0.5735; Testing Accuracy: 80.82%

	Epoch: 6
	Training Loss: 1.0006; Training Accuracy: 82.746%
	Testing Loss: 0.6144; Testing Accuracy: 79.93%

	Epoch: 7
	Training Loss: 0.9319; Training Accuracy: 83.926%
	Testing Loss: 0.5296; Testing Accuracy: 81.96%

	Epoch: 8
	Training Loss: 0.8593; Training Accu

### Experiment 11

In [19]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 32)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 32)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.8182; Training Accuracy: 48.626%
	Testing Loss: 1.0154; Testing Accuracy: 63.65%

	Epoch: 1
	Training Loss: 1.8735; Training Accuracy: 66.82%
	Testing Loss: 0.8157; Testing Accuracy: 71.64%

	Epoch: 2
	Training Loss: 1.4811; Training Accuracy: 74.1%
	Testing Loss: 0.7774; Testing Accuracy: 73.24%

	Epoch: 3
	Training Loss: 1.2552; Training Accuracy: 78.01%
	Testing Loss: 0.6007; Testing Accuracy: 79.32%

	Epoch: 4
	Training Loss: 1.1042; Training Accuracy: 80.826%
	Testing Loss: 0.5966; Testing Accuracy: 79.53%

	Epoch: 5
	Training Loss: 0.9851; Training Accuracy: 83.006%
	Testing Loss: 0.5544; Testing Accuracy: 81.94%

	Epoch: 6
	Training Loss: 0.9019; Training Accuracy: 84.374%
	Testing Loss: 0.5175; Testing Accuracy: 82.48%

	Epoch: 7
	Training Loss: 0.8294; Training Accuracy: 85.738%
	Testing Loss: 0.4715; Testing Accuracy: 83.76%

	Epoch: 8
	Training Loss: 0.7653; Training Accu

### Experiment 12

In [20]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 32)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 32)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.8845; Training Accuracy: 47.072%
	Testing Loss: 1.0529; Testing Accuracy: 62.12%

	Epoch: 1
	Training Loss: 1.9713; Training Accuracy: 64.99%
	Testing Loss: 0.875; Testing Accuracy: 69.99%

	Epoch: 2
	Training Loss: 1.5457; Training Accuracy: 72.974%
	Testing Loss: 0.6873; Testing Accuracy: 75.81%

	Epoch: 3
	Training Loss: 1.2962; Training Accuracy: 77.468%
	Testing Loss: 0.7704; Testing Accuracy: 75.19%

	Epoch: 4
	Training Loss: 1.1413; Training Accuracy: 80.47%
	Testing Loss: 0.5555; Testing Accuracy: 80.76%

	Epoch: 5
	Training Loss: 1.0165; Training Accuracy: 82.35%
	Testing Loss: 0.5385; Testing Accuracy: 81.41%

	Epoch: 6
	Training Loss: 0.9295; Training Accuracy: 83.906%
	Testing Loss: 0.4685; Testing Accuracy: 84.07%

	Epoch: 7
	Training Loss: 0.8357; Training Accuracy: 85.664%
	Testing Loss: 0.4521; Testing Accuracy: 84.77%

	Epoch: 8
	Training Loss: 0.7855; Training Accu

### Experiment 13

In [21]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [1, 1, 1, 1])
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 32)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 32)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.1043; Training Accuracy: 43.454%
	Testing Loss: 1.2535; Testing Accuracy: 55.07%

	Epoch: 1
	Training Loss: 1.9813; Training Accuracy: 64.834%
	Testing Loss: 0.885; Testing Accuracy: 69.36%

	Epoch: 2
	Training Loss: 1.5217; Training Accuracy: 73.414%
	Testing Loss: 0.8014; Testing Accuracy: 73.12%

	Epoch: 3
	Training Loss: 1.2719; Training Accuracy: 77.956%
	Testing Loss: 0.5813; Testing Accuracy: 80.06%

	Epoch: 4
	Training Loss: 1.1106; Training Accuracy: 80.854%
	Testing Loss: 0.5861; Testing Accuracy: 80.28%

	Epoch: 5
	Training Loss: 0.9904; Training Accuracy: 82.72%
	Testing Loss: 0.5355; Testing Accuracy: 81.97%

	Epoch: 6
	Training Loss: 0.8874; Training Accuracy: 84.78%
	Testing Loss: 0.5227; Testing Accuracy: 82.47%

	Epoch: 7
	Training Loss: 0.8065; Training Accuracy: 86.124%
	Testing Loss: 0.4773; Testing Accuracy: 84.06%

	Epoch: 8
	Training Loss: 0.7527; Training Acc

### Experiment 11

In [134]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 32)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 32)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.0979; Training Accuracy: 43.474%
	Testing Loss: 1.6336; Testing Accuracy: 48.3%

	Epoch: 1
	Training Loss: 1.9974; Training Accuracy: 64.442%
	Testing Loss: 0.7966; Testing Accuracy: 71.92%

	Epoch: 2
	Training Loss: 1.5179; Training Accuracy: 73.536%
	Testing Loss: 0.7326; Testing Accuracy: 75.39%

	Epoch: 3
	Training Loss: 1.2711; Training Accuracy: 78.002%
	Testing Loss: 0.6211; Testing Accuracy: 79.11%

	Epoch: 4
	Training Loss: 1.107; Training Accuracy: 80.898%
	Testing Loss: 0.519; Testing Accuracy: 82.51%

	Epoch: 5
	Training Loss: 0.9796; Training Accuracy: 83.05%
	Testing Loss: 0.5409; Testing Accuracy: 82.09%

	Epoch: 6
	Training Loss: 0.887; Training Accuracy: 84.758%
	Testing Loss: 0.4434; Testing Accuracy: 84.98%

	Epoch: 7
	Training Loss: 0.8071; Training Accuracy: 86.118%
	Testing Loss: 0.4536; Testing Accuracy: 85.52%

	Epoch: 8
	Training Loss: 0.7435; Training Accur

In [31]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

11173962

In [22]:
torch.cuda.empty_cache()